In [10]:

import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning as L

<module 'torch.optim' from 'c:\\Users\\revil\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\torch\\optim\\__init__.py'>


In [18]:

encoder = nn.Sequential(nn.Linear(28*28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28*28))

class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        x, _ = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        self.log("training_loss", loss)
        return loss
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
autoencoder = LitAutoEncoder(encoder, decoder)

In [14]:
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

In [25]:
trainer = L.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(autoencoder, train_dataloaders=train_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | encoder | Sequential | 50.4 K | eval 
1 | decoder | Sequential | 51.2 K | train
-----------------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407     Total estimated model params size (MB)
4         Modules in train mode
4         Modules in eval mode
c:\Users\revil\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [26]:
import torch
checkpoint = "./lightning_logs/version_1/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

encoder = autoencoder.encoder
encoder.eval()

fake_image_batch = torch.rand(4, 28*28, device=autoencoder.device)
embeddings = encoder(fake_image_batch)
print(embeddings)

tensor([[ 0.3866, -0.5344, -0.1257],
        [ 0.4922, -0.6583, -0.1314],
        [ 0.3424, -0.5843, -0.1445],
        [ 0.3913, -0.5754, -0.0768]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
